# Assignment2

#### 2. Import any Python packages that you will need for this activity.

In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
%matplotlib inline

### Open the file into a data frame
#### 3. Open the insurance.csv file into a Pandas data frame and verify the contents of the file by displaying the first 10 rows.

In [97]:
dat_insurance = pd.read_csv('insurance.csv')

In [98]:
dat_insurance.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


### Create a list of numeric features

In [99]:
numeric_features = ['age','bmi','children']

In [100]:
dat_insurance[numeric_features]

,age,bmi,children
0,19,27.900,0
1,18,33.770,1
2,28,33.000,3
3,33,22.705,0
4,32,28.880,0
...,...,...,...
1333,50,30.970,3
1334,18,31.920,0
1335,18,36.850,0
1336,21,25.800,0


### Get a correlation table on the numeric features

In [101]:
dat_insurance[numeric_features].corr()

,age,bmi,children
age,1.000000,0.109272,0.042469
bmi,0.109272,1.000000,0.012759
children,0.042469,0.012759,1.000000


#### All are correlated positively, but not highly correlated.

### Recode the target Variable (charges) and rename as 'charges_code' 0 = low and 1 = high

#### 4. Modify the data frame by discretizing the charges feature such that charges over $10,000 are designated as high and those at this level and below are designated as low. This will give you a data set for a two-bin binary classification.

In [102]:
# Discretizing the 'charges' column using pd.cut with the correct bins and labels
dat_insurance['charges_code'] = pd.cut(dat_insurance['charges'],
                                      bins=[0, 10000, float('inf')], 
                                      include_lowest=True, 
                                      labels=[0, 1])
dat_insurance.head(10)

#add 1, 0 for charges > 10000
#charges_code 1 = High and low = 0
# dat_insurance['charges_code'] = np.where(dat_insurance['charges'] >= 10000, 1, 0)
# dat_insurance.head(10)

,age,sex,bmi,children,smoker,region,charges,charges_code
0,19,female,27.900,0,yes,southwest,16884.92400,1
1,18,male,33.770,1,no,southeast,1725.55230,0
2,28,male,33.000,3,no,southeast,4449.46200,0
3,33,male,22.705,0,no,northwest,21984.47061,1
4,32,male,28.880,0,no,northwest,3866.85520,0
5,31,female,25.740,0,no,southeast,3756.62160,0
6,46,female,33.440,1,no,southeast,8240.58960,0
7,37,female,27.740,3,no,northwest,7281.50560,0
8,37,male,29.830,2,no,northeast,6406.41070,0
9,60,female,25.840,0,no,northwest,28923.13692,1


In [103]:
dat_insurance.drop('charges', axis=1, inplace=True)
dat_insurance.head(10)

,age,sex,bmi,children,smoker,region,charges_code
0,19,female,27.900,0,yes,southwest,1
1,18,male,33.770,1,no,southeast,0
2,28,male,33.000,3,no,southeast,0
3,33,male,22.705,0,no,northwest,1
4,32,male,28.880,0,no,northwest,0
5,31,female,25.740,0,no,southeast,0
6,46,female,33.440,1,no,southeast,0
7,37,female,27.740,3,no,northwest,0
8,37,male,29.830,2,no,northeast,0
9,60,female,25.840,0,no,northwest,1


### Get Correlations for all numeric values and the target

In [104]:
target_feature = list(['charges_code'])
print(target_feature)
dat_insurance[target_feature]


['charges_code']


,charges_code
0,1
1,0
2,0
3,1
4,0
...,...
1333,1
1334,0
1335,0
1336,0


In [105]:
dat_insurance[numeric_features+target_feature].corr()

<ipython-input-105-ded669ada026>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  dat_insurance[numeric_features+target_feature].corr()


,age,bmi,children
age,1.000000,0.109272,0.042469
bmi,0.109272,1.000000,0.012759
children,0.042469,0.012759,1.000000


#### There is a moderate positive correlation between age and the charges_code. This makes sense, higher age, likely higher income charges, and cost of treatment based on timespand of life. BMI and Childeren doen't seem to affect the indiviuals insuarance charges too much.

### Aggregate numeric values by nominal

In [106]:
dat_insurance.groupby('bmi').mean()

<ipython-input-106-732e25d1b8b6>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dat_insurance.groupby('bmi').mean()


,age,children
bmi,,
15.960,18.000000,0.000000
16.815,29.500000,1.500000
17.195,26.000000,2.000000
17.290,27.666667,1.333333
17.385,23.000000,1.000000
...,...,...
48.070,46.000000,2.000000
49.060,58.000000,0.000000
50.380,23.000000,1.000000


In [107]:
dat_insurance.groupby('children').mean()

<ipython-input-107-d6e804627490>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dat_insurance.groupby('children').mean()


,age,bmi
children,,
0,38.444251,30.550296
1,39.453704,30.623148
2,39.445833,30.978104
3,41.566879,30.684331
4,39.000000,31.391200
5,35.611111,29.605000


In [108]:
dat_insurance.groupby('sex').mean()

<ipython-input-108-a7bf059d7bab>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dat_insurance.groupby('sex').mean()


,age,bmi,children
sex,,,
female,39.503021,30.377749,1.074018
male,38.917160,30.943129,1.115385


In [109]:
dat_insurance.groupby('age').mean()

<ipython-input-109-14c4433d93a9>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dat_insurance.groupby('age').mean()


,bmi,children
age,,
18,31.326159,0.449275
19,28.596912,0.426471
20,30.632759,0.862069
21,28.185714,0.785714
22,31.087679,0.714286
23,31.454464,1.000000
24,29.142679,0.464286
25,29.693929,1.285714
26,29.428929,1.071429


##### Recode 'sex' column as Male = 1; rename column to 'male'
##### Recode 'smoker' column as Yes = 1; rename to 'smoker_code'
##### Recode region's using get.dummies or np.where 'region name'
<!-- age                int64
sex               object
bmi              float64
children           int64
smoker            object
region            object
charges_code    category
dtype: object -->
#### 5. Create dummies as necessary for any categorical features in the data set.

In [110]:
#dummy variables for 'sex', 'smoker', and 'region'
#create column names and fill values  based on conditons matching values 1 = true,0=false(else)
dat_insurance['male'] = np.where(dat_insurance['sex'] == 'male', 1, 0)
dat_insurance['smoker_code'] = np.where(dat_insurance['smoker'] == 'yes', 1, 0)

#region dummy variables
df_dummies = pd.get_dummies(dat_insurance, columns=['region'])
dat_insurance = df_dummies.drop(['region_southwest'], axis=1)
# dat_insurance['region_northeast'] = np.where(dat_insurance['region'] == 'northeast', 1, 0)
# dat_insurance['region_northwest'] = np.where(dat_insurance['region'] == 'northwest', 1, 0)
# dat_insurance['region_southeast'] = np.where(dat_insurance['region'] == 'southeast', 1, 0)
# dat_insurance['region_southwest'] = np.where(dat_insurance['region'] == 'southwest', 1, 0)
dat_insurance.head(10)

,age,sex,bmi,children,smoker,charges_code,male,smoker_code,region_northeast,region_northwest,region_southeast
0,19,female,27.900,0,yes,1,0,1,0,0,0
1,18,male,33.770,1,no,0,1,0,0,0,1
2,28,male,33.000,3,no,0,1,0,0,0,1
3,33,male,22.705,0,no,1,1,0,0,1,0
4,32,male,28.880,0,no,0,1,0,0,1,0
5,31,female,25.740,0,no,0,0,0,0,0,1
6,46,female,33.440,1,no,0,0,0,0,0,1
7,37,female,27.740,3,no,0,0,0,0,1,0
8,37,male,29.830,2,no,0,1,0,1,0,0
9,60,female,25.840,0,no,1,0,0,0,1,0


In [111]:
dat_insurance.drop(['smoker', 'sex'], axis=1, inplace=True)
dat_insurance.head(10)

,age,bmi,children,charges_code,male,smoker_code,region_northeast,region_northwest,region_southeast
0,19,27.900,0,1,0,1,0,0,0
1,18,33.770,1,0,1,0,0,0,1
2,28,33.000,3,0,1,0,0,0,1
3,33,22.705,0,1,1,0,0,1,0
4,32,28.880,0,0,1,0,0,1,0
5,31,25.740,0,0,0,0,0,0,1
6,46,33.440,1,0,0,0,0,0,1
7,37,27.740,3,0,0,0,0,1,0
8,37,29.830,2,0,1,0,1,0,0
9,60,25.840,0,1,0,0,0,1,0


### Create list of training features.  Then code x and Y as Training and Target

#### 6. Break the data frame into features and target frames, then create training and testing partitions with a test size of 30% and a random state of 101. Test the means of the charges bins to ensure consistent distribution between partitions.

In [112]:
dat_insurance

,age,bmi,children,charges_code,male,smoker_code,region_northeast,region_northwest,region_southeast
0,19,27.900,0,1,0,1,0,0,0
1,18,33.770,1,0,1,0,0,0,1
2,28,33.000,3,0,1,0,0,0,1
3,33,22.705,0,1,1,0,0,1,0
4,32,28.880,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,1,1,0,0,1,0
1334,18,31.920,0,0,0,0,1,0,0
1335,18,36.850,0,0,0,0,0,0,1
1336,21,25.800,0,0,0,0,0,0,0


In [113]:
training_features = list(['age','bmi','children','male','smoker_code','region_northeast','region_northwest','region_southeast'])
X=dat_insurance[training_features]
y=dat_insurance[target_feature]

### Split training and testing sets, then validate distribution.

In [114]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=101)

In [115]:
y_train.mean()

<ipython-input-115-46c402737bb5>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  y_train.mean()


Series([], dtype: float64)

In [116]:
y_test.mean()

<ipython-input-116-34581e62523f>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  y_test.mean()


Series([], dtype: float64)

##### Naïve Bayes Classification
##### Create a Naïve Bayes classification model on the target feature (charges bins). Compare the relative performance of the training and testing partitions and provide your analysis of the results, including accuracy and error rates.

### Using a Naive Bayes Classifier

In [117]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(X_train, y_train)

C:\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [118]:
# Generate predictions based on the model
y_train_pred_nb = nb.predict(X_train)

# Report the 'actual' vs 'prediction' in a confusion matrix
cm = np.array(confusion_matrix(y_train, y_train_pred_nb, labels=[1,0]))
confusion = pd.DataFrame(cm, index=['high_charges', 'low_charges'],
                         columns=['predicted_high_charges','predicted_low_charges'])
print(confusion)
print()
print(classification_report(y_train, y_train_pred_nb)) 

accuracy = accuracy_score(y_true=y_train, y_pred=y_train_pred_nb)
accuracy.round(3)

              predicted_high_charges  predicted_low_charges
high_charges                     203                    241
low_charges                        0                    492

              precision    recall  f1-score   support

           0       0.67      1.00      0.80       492
           1       1.00      0.46      0.63       444

    accuracy                           0.74       936
   macro avg       0.84      0.73      0.72       936
weighted avg       0.83      0.74      0.72       936



0.743

### Check accuracy of Test Set

In [119]:
y_test_pred_nb = nb.predict(X_test)
confusion_matrix(y_test,y_test_pred_nb)

# Report the 'actual' vs 'prediction' in a confusion matrix
cm = np.array(confusion_matrix(y_test, y_test_pred_nb, labels=[1,0]))
confusion = pd.DataFrame(cm, index=['high_charges', 'low_charges'],
                         columns=['predicted_high_charges','predicted_low_charges'])
print(confusion)
print()
print(classification_report(y_test, y_test_pred_nb)) 

accuracy = accuracy_score(y_true=y_test, y_pred=y_test_pred_nb)
accuracy.round(3)

              predicted_high_charges  predicted_low_charges
high_charges                      71                    111
low_charges                        0                    220

              precision    recall  f1-score   support

           0       0.66      1.00      0.80       220
           1       1.00      0.39      0.56       182

    accuracy                           0.72       402
   macro avg       0.83      0.70      0.68       402
weighted avg       0.82      0.72      0.69       402



0.724

#### The model showed perfect precision for predicting high charges but a relatively low recall 46%, which means that it's not likely good at predicting every high charge and can potentially miss them. Also, the overall accuracy didn't preform the greatest at 74.3 percent for the training model and 72.4 percent for the test model, albeit they're close to eacher in scores. Both training and test model had the same perfect high charge prediction, which I think is due to being able to always predict ages 55 and greater based on the mean of the ages when grouped by all data like charges greater than 10,000 being = 1 for that age demographic. 

### Using Logistic Regression

In [120]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train, y_train)

C:\anaconda3\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [121]:
# Generate predictions based on the model
y_train_pred_log = log.predict(X_train)

# Report the 'actual' vs 'prediction' in a confusion matrix
cm = np.array(confusion_matrix(y_train, y_train_pred_log, labels=[1,0]))
confusion = pd.DataFrame(cm, index=['high_charges', 'low_charges'],
                         columns=['predicted_high_charges','predicted_low_charges'])
print(confusion)
print()
print(classification_report(y_train, y_train_pred_log)) 

accuracy = accuracy_score(y_true=y_train, y_pred=y_train_pred_log)
accuracy.round(3)

              predicted_high_charges  predicted_low_charges
high_charges                     400                     44
low_charges                       37                    455

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       492
           1       0.92      0.90      0.91       444

    accuracy                           0.91       936
   macro avg       0.91      0.91      0.91       936
weighted avg       0.91      0.91      0.91       936



0.913

##### Precision and recall show that both are pretty accurate in predicting also indicated by the low unscusfull predictions in the confusion matrix. This near similar preciions and recall score is likely attributed to the near 50/50 split in ssupport data between low and high charges.

In [122]:
print('intercept: {}'.format(log.intercept_))
print('coefficients (log odds) (age,bmi,children,male,smoker_code,northwest,southeast,southwest): {}'.format(log.coef_))
print('coefficients (odds ratio) (age,bmi,children,male,smoker_code,northwest,southeast,southwest): {}'.format(np.exp(log.coef_)))


intercept: [-7.18933405]
coefficients (log odds) (age,bmi,children,male,smoker_code,northwest,southeast,southwest): [[ 0.12971956  0.02355095  0.02414119 -0.17852994  5.89013363  0.48534313
   0.49230526  0.2754188 ]]
coefficients (odds ratio) (age,bmi,children,male,smoker_code,northwest,southeast,southwest): [[  1.13850906   1.02383046   1.02443495   0.83649901 361.45358185
    1.6247324    1.63608348   1.31708215]]


In [123]:
##### Log Odds:
##### A 1 year increae in age causes the log odds ratio of high charges to increase by 0.1298

##### odds ratio:
##### For each additional year of age, the odds of high charges increase by about 13.9 %.
##### Being a male would also reduce your charges and being from the southeast and,southwest.

### Check accuracy of Test Set

In [124]:
# Generate predictions based on the model
y_test_pred_log = log.predict(X_test)

# Report the 'actual' vs 'prediction' in a confusion matrix
cm = np.array(confusion_matrix(y_test, y_test_pred_log, labels=[1,0]))
confusion = pd.DataFrame(cm, index=['high_charges', 'low_charges'],
                         columns=['predicted_high_charges','predicted_low_charges'])
print(confusion)
print()
print(classification_report(y_test, y_test_pred_log)) 

accuracy = accuracy_score(y_true=y_test, y_pred=y_test_pred_log)
accuracy.round(3)

              predicted_high_charges  predicted_low_charges
high_charges                     162                     20
low_charges                       36                    184

              precision    recall  f1-score   support

           0       0.90      0.84      0.87       220
           1       0.82      0.89      0.85       182

    accuracy                           0.86       402
   macro avg       0.86      0.86      0.86       402
weighted avg       0.86      0.86      0.86       402



0.861

##### Summary:Naive Bayes may be leveraging the age feature effectively to predict high charges, especially for older individuals, as age is moderately correlated with whether someone has high insurance charges or not. We can further see why based on grouping the data by the mean age. From age 55 and above, the average charges_code is 1.0, meaning 100% of individuals in this age range have high charges. This may be the reason that the model is less stable at predicting the low charges here and that it has a low recall and might struggle to predict high charges for those younger individuals. However overall accuracy is lower than the Logistic model, by a fair amount that it would be worth using the Logisitc Model for predicting the high and low charges.